# **Jornada de Dados <Laboratória>**
# **PROJ005**


A VAREJISTA, empresa líder no setor, enfrenta o desafio de gerenciar grandes volumes de dados dispersos. Para aprimorar a tomada de decisões, está sendo implementado um sistema ETL robusto com tabelas fato e dimensões. O projeto visa extrair, transformar e carregar dados de forma eficiente, estruturando-os hierarquicamente para facilitar análises estratégicas. Além de otimizar o armazenamento, a iniciativa busca ampliar a capacidade da empresa de identificar padrões e oportunidades de mercado, respondendo com agilidade às mudanças na demanda do consumidor.






